In [ ]:
import pandas as pd
import numpy as np
import requests
from dotenv import load_dotenv
import sys 
from pathlib import Path
import os

# ------------------------------ #
# Must run together to access NASS API key
# ------------------------------ #
src_path = Path.cwd() / 'src'
sys.path.insert(0, str(src_path))
from utils.nass_api import NASS_API_KEY
# ------------------------------ #

In [ ]:
base_url = "https://quickstats.nass.usda.gov/api/api_GET/"

params = {'key': NASS_API_KEY,
          'source_desc': 'SURVEY',
          'sector_desc': 'CROPS',
          'group_desc': 'FIELD CROPS',
          'commodity_desc': 'CORN',
          'statisticcat_desc': 'AREA HARVESTED',
          'short_desc': 'CORN, GRAIN - ACRES HARVESTED',
          'agg_level_desc': 'STATE'
         }

response = requests.get(base_url, params=params)

corn_ac = response.json()

corn_ac = pd.DataFrame(corn_ac['data'])

### Prepare annual corn harvest data (acres harvested) by state

In [ ]:
select_vars = ['year', 'state_name', 'reference_period_desc', 'freq_desc',  
               'unit_desc', 'sector_desc',  'class_desc', 'commodity_desc', 
               'short_desc', 'group_desc', 'statisticcat_desc', 'Value']

corn_ac_sel = corn_ac.loc[:, select_vars]

year_filter = corn_ac_sel['reference_period_desc'] == 'YEAR'
other_states_filter = corn_ac_sel['state_name'] != 'OTHER STATES'

corn_ac_sel = corn_ac_sel.loc[(year_filter) & (other_states_filter), ]

#### Check if there are any missing values in columns anywhere. 

In [ ]:
corn_ac_sel.isna().sum()

In [ ]:
corn_ac_sel['Value'].isin(["(D)", "(Z)"]).any()

### Save to CSV

In [ ]:
# Save csv
corn_ac_sel.to_csv('data/corn_acres_harvested_by_state.csv', index=False)